In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip "Path_to_your_zip" -d "Where_to_put_it"
!unzip /content/drive/MyDrive/tumor_images.zip -d /content/brain_tumor_dataset

Archive:  /content/drive/MyDrive/tumor_images.zip
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/1 no.jpeg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/10 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/11 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/12 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/13 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/14 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/15 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/17 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/18 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/19 no.jpg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/2 no.jpeg  
  inflating: /content/brain_tumor_dataset/brain_tumor_dataset/no/20 no.jpg  
  inflating: /content/brai

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
from google.colab import drive, files

# --- 1. CONNECT TO DRIVE ---
drive.mount('/content/drive')

# --- 2. DATASET PATH ---
DATA_PATH = '/content/brain_tumor_dataset/brain_tumor_dataset'

# --- 3. LOAD AND PREPROCESS DATA ---
IMG_SIZE = (150, 150)
BATCH_SIZE = 32

# Creating Training and Validation Sets
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_PATH,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

# Rescale pixel values (0-255 to 0-1)
normalization_layer = layers.Rescaling(1./255)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Optimize performance
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# --- 4. BUILD THE CNN MODEL ---
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid') # Binary Output
])

# --- 5. COMPILE AND TRAIN ---
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("\n--- Starting Training ---")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10
)

# --- 6. SAVE AND DOWNLOAD ---
model.save('brain_tumor_model.h5')
print("\n--- Model Saved as 'brain_tumor_model.h5' ---")
files.download('brain_tumor_model.h5')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 253 files belonging to 2 classes.
Using 203 files for training.
Found 253 files belonging to 2 classes.
Using 50 files for validation.

--- Starting Training ---
Epoch 1/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 13s 2s/step - accuracy: 0.6606 - loss: 0.8483 - val_accuracy: 0.7800 - val_loss: 0.5418
Epoch 2/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.8007 - loss: 0.4825 - val_accuracy: 0.8200 - val_loss: 0.5317
Epoch 3/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 20s 1s/step - accuracy: 0.8394 - loss: 0.3950 - val_accuracy: 0.8000 - val_loss: 0.5268
Epoch 4/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.8137 - loss: 0.3905 - val_accuracy: 0.8200 - val_loss: 0.5392
Epoch 5/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 9s 1s/step - accuracy: 0.8698 - loss: 0.3208 - val_accuracy: 0.8600 - val_loss: 0.5275
Epoch 6/10
7/7 ━━━━━━━━━━━━━━━━━━━━ 11s 2s/step - accuracy: 0.8752 - loss: 0.3016 - 


--- Model Saved as 'brain_tumor_model.h5' ---


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>